In [1]:
 !pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from IPython.display import display

In [3]:
spark_application_name = "Spark_Application_Name"

In [4]:
spark = (SparkSession.builder.appName(spark_application_name).getOrCreate())

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/16 16:44:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/16 16:44:19 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
22/06/16 16:44:19 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
22/06/16 16:44:19 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
22/06/16 16:44:19 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
22/06/16 16:44:19 WARN 

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.net.BindException: Cannot assign requested address: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at java.base/sun.nio.ch.Net.bind0(Native Method)
	at java.base/sun.nio.ch.Net.bind(Net.java:459)
	at java.base/sun.nio.ch.Net.bind(Net.java:448)
	at java.base/sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:227)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:134)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:562)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1334)
	at io.netty.channel.AbstractChannelHandlerContext.invokeBind(AbstractChannelHandlerContex

In [6]:
amazon_csv = "stocks_data/AMAZON.csv"
apple_csv = "stocks_data/APPLE.csv"
facebook_csv = "stocks_data/FACEBOOK.csv"
google_csv = "stocks_data/GOOGLE.csv"
microsoft_csv = "stocks_data/MICROSOFT.csv"
tesla_csv = "stocks_data/TESLA.csv"
zoom_csv = "stocks_data/ZOOM.csv"

unclean_df_amazon = spark.read.csv(amazon_csv, header="true", inferSchema="true", multiLine="true", escape='"')
unclean_df_apple = spark.read.csv(apple_csv, header="true", inferSchema="true", multiLine="true", escape='"')
unclean_df_facebook = spark.read.csv(facebook_csv, header="true", inferSchema="true", multiLine="true", escape='"')
unclean_df_google = spark.read.csv(google_csv, header="true", inferSchema="true", multiLine="true", escape='"')
unclean_df_microsoft = spark.read.csv(microsoft_csv, header="true", inferSchema="true", multiLine="true", escape='"')
unclean_df_tesla = spark.read.csv(tesla_csv, header="true", inferSchema="true", multiLine="true", escape='"')
unclean_df_zoom = spark.read.csv(zoom_csv, header="true", inferSchema="true", multiLine="true", escape='"')

NameError: name 'spark' is not defined

In [25]:
from functools import reduce
from pyspark.sql import DataFrame

dfs = [unclean_df_amazon, unclean_df_apple, unclean_df_facebook, unclean_df_google, unclean_df_microsoft, unclean_df_tesla, unclean_df_zoom]
df = reduce(DataFrame.union, dfs)

In [26]:
df.select("company_name").distinct().show()

+------------+
|company_name|
+------------+
|      AMAZON|
|       APPLE|
|    FACEBOOK|
|      GOOGLE|
|   MICROSOFT|
|       TESLA|
|        ZOOM|
+------------+



In [41]:
display(df)

DataFrame[Date: string, High: double, Low: double, Open: double, Close: double, Volume: double, Adj Close: double, company_name: string]

### Data Cleansing

In [31]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- company_name: string (nullable = true)



In [43]:
from pyspark.sql.functions import *

df = df.withColumn("Date", to_date(col("Date"), "yyyy-MM-dd"))
df.show()

+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+------------+
|      Date|             High|              Low|             Open|            Close|   Volume|        Adj Close|company_name|
+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+------------+
|2017-01-03| 758.760009765625|747.7000122070312|757.9199829101562|753.6699829101562|3521100.0|753.6699829101562|      AMAZON|
|2017-01-04|759.6799926757812|754.2000122070312|758.3900146484375|757.1799926757812|2510500.0|757.1799926757812|      AMAZON|
|2017-01-05|782.4000244140625| 760.260009765625|761.5499877929688|780.4500122070312|5830100.0|780.4500122070312|      AMAZON|
|2017-01-06|799.4400024414062|  778.47998046875|782.3599853515625| 795.989990234375|5986200.0| 795.989990234375|      AMAZON|
|2017-01-09|  801.77001953125|  791.77001953125|            798.0|796.9199829101562|3446100.0|796.9199829101562|      

In [44]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- company_name: string (nullable = true)



### NULLS

In [47]:
noNullsDF = df.na.fill(value=0, subset=["Volume"])
noNullsDF.show()

# We can see that there are 987 Volume columns in apple, microsoft and tesla DFs have null values.
# We do not want to remove every rows that contain a null value because if we do so 987 rows in 3 DFs will be removed,
# which also means a lot of data from other columns of that row will be lost, and these data is important in the analytics of other information of that day

+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+------------+
|      Date|             High|              Low|             Open|            Close|   Volume|        Adj Close|company_name|
+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+------------+
|2017-01-03| 758.760009765625|747.7000122070312|757.9199829101562|753.6699829101562|3521100.0|753.6699829101562|      AMAZON|
|2017-01-04|759.6799926757812|754.2000122070312|758.3900146484375|757.1799926757812|2510500.0|757.1799926757812|      AMAZON|
|2017-01-05|782.4000244140625| 760.260009765625|761.5499877929688|780.4500122070312|5830100.0|780.4500122070312|      AMAZON|
|2017-01-06|799.4400024414062|  778.47998046875|782.3599853515625| 795.989990234375|5986200.0| 795.989990234375|      AMAZON|
|2017-01-09|  801.77001953125|  791.77001953125|            798.0|796.9199829101562|3446100.0|796.9199829101562|      

In [ ]:
# Don't need to impute cast to doubles cause already done
# Some of airbnb data was BS so had to be imputed, but here it s fr so we shouldn t remove extreme values, car elles
# sont veridiques.

In [50]:
outputPath = "sf-stocks-clean.parquet"

noNullsDF = noNullsDF.withColumnRenamed("Adj Close", "AdjClose")
noNullsDF.write.mode("overwrite").parquet(outputPath)


22/06/13 00:49:05 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
